In [ ]:
# Import the libraries

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, io, img_as_ubyte
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu

In [ ]:
class WesternBlotQuantification():
    
    CONTROL = {}
    EXPERIMENTAL = {}
    
    def __init__(self, conc, blot_img, control = None):
        """
        concs: List, a list of concentrations (or different timing) used in the experiment.
        blot_img: Image of the blot.
        control: Image of a blot for normalazation. It could be GAPDH or B Actin or something else.
        If total measurements are not required, do not add it. By defautl, its value is none.
        """
        self.conc = conc
        self.blot_img = blot_img
        self.control = control
        
    def read_gray(self):
        """
        Reads the blot image as grey.
        """
        if self.blot_img.split(".")[-1] == "jpg":
            return img_as_ubyte(rgb2gray(io.imread(self.blot_img)))
    
        else:
            return img_as_ubyte(rgb2gray(rgba2rgb(io.imread(self.blot_img))))
    
    def show_gray(self):
        """
        Displays the blot image in grey format.
        """
        plt.imshow(self.read_gray())
        plt.show()
        
    def thresh_img(self):
        """
        Thresholds the grey image with threshod otsu algorithm.
        """
        segmented = self.read_gray() < threshold_otsu(self.read_gray())
        return segmented
    
    def show_thresh_img(self):
        """
        Displays the thresholded image.
        """
        plt.imshow(self.thresh_img())
        plt.show()
        
    def band_labeling(self):
        """
        Labels the objects in the image.
        """
        return measure.label(self.thresh_img(), connectivity = self.read_gray().ndim) 
    
    def show_labeled(self):
        """
        Displays the labeled image.
        """
        plt.imshow(self.band_labeling())
        plt.show()
        
    def band_separation(self):
        """
        Isolates the objects in the image.
        """
        col_img = np.any(self.thresh_img(), axis = 0)
        separated_obj = np.flatnonzero(np.diff(col_img))
        return separated_obj
    
    def band_measurement(self):
        """
        Measure the area of each object in the image.
        """
        start = 0
        end = 1
        band_location = self.band_separation()
        original_img = self.read_gray()
        labeled_img = self.band_labeling()
        
        for num in range(len(band_location) // 2):
            properties = measure.regionprops(labeled_img[:, band_location[start]: band_location[end]], 
                                             original_img[:, band_location[start]: band_location[end]])

            if len(properties) == 0:
                raise Exception("No object detected! Check the image.")

            elif len(properties) == 1:
                for ar in properties:
                    if self.control == None:
                        WesternBlotQuantification.EXPERIMENTAL[self.conc[num]] = ar.area
                        
                    else:
                        WesternBlotQuantification.CONTROL[self.conc[num]] = ar.area

            else:
                final = max([ar.area for ar in properties])
                if self.control == None:
                    WesternBlotQuantification.EXPERIMENTAL[self.conc[num]] = final
                        
                else:
                    WesternBlotQuantification.CONTROL[self.conc[num]] = final

            start += 2
            end += 2
            
        if self.control == None:
            return WesternBlotQuantification.EXPERIMENTAL
        
        else:
            return WesternBlotQuantification.CONTROL
        
    def total_measurement(self):
        """
        Takes band measurements of an antibody and its control, divides each bands area to its corresponding control area
        """
    
        return {pair[0]: int(pair[1]) / int(pair[2]) for pair in zip(WesternBlotQuantification.EXPERIMENTAL.keys(), 
                                                                     WesternBlotQuantification.EXPERIMENTAL.values(), 
                                                                     WesternBlotQuantification.CONTROL.values())}
    
    def wb_graph(self, title, x_label, y_label, img_name, display = False):
    """
    Creates and saves graphs based on the info from band_measurement function.
    """
        self.title = title
        self.x_label = x_label
        self.y_label = y_label
        self.img_name = img_name
        self.display = display
        
        if self.display == False:
            plt.figure(figsize = (10, 7))
            plt.bar(WesternBlotQuantification.EXPERIMENTAL.keys(), WesternBlotQuantification.EXPERIMENTAL.values())             
            plt.xlabel(self.x_label)
            plt.ylabel(self.y_label)
            plt.title(self.title) 
            plt.savefig(self.img_name)
            
        else:
            plt.figure(figsize = (10, 7))
            plt.bar(WesternBlotQuantification.EXPERIMENTAL.keys(), WesternBlotQuantification.EXPERIMENTAL.values())             
            plt.xlabel(self.x_label)
            plt.ylabel(self.y_label)
            plt.title(self.title) 
            plt.show()